In [ ]:
import numpy as np
import pandas as pd
import random
import copy
from itertools import product
from scipy import stats 
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Helper Functions

In [ ]:
# modified combinations with replacement function so results include those which are not sorted 
def combinations_with_replacement(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in product(range(n), repeat=r):
        yield tuple(pool[i] for i in indices)

In [ ]:
# returns a combined list with no repeats 
def combined_list(listA, listB):
    set1 = set(listA)
    set2 = set(listB)
    
    combined = listA + list(set2 - set1)
    return combined

In [ ]:
# returns all possible appointment schedules given N patients and F slots, where each slot is assigned at least one patient      
def all_schedules(N, F):
    A1 = []
    slots = (range(1, F+1))
    possibilities = list(combinations_with_replacement(slots, N))
    for p in possibilities: 
        p = list(p)
        if (all (x in p for x in list(slots))):
            A1.append(p)
    return A1

In [ ]:
# transform schedule into dictionary
def transform_schedule(schedule):
    schedule_dict = {}
    for i in range(len(schedule)):
        if schedule[i] not in schedule_dict:
            schedule_dict[schedule[i]] = list()
        schedule_dict[schedule[i]].append(i)
    return (schedule_dict)

In [ ]:
# load dataset from previous program 
sample_dataset = np.load('ss_SMOTE_simulation_dataset.npy')

# helper function that will randomly select N patients from the dataset 
# returns a list of N show probabilities, from least to greatest 
def patient_selection(N):
    indices = np.random.choice(len(sample_dataset), N, replace = True)
    selected = sample_dataset[indices]
    
    # sort everything by the show probabilities in increasing order
    sorted_list = selected[selected[:, 2].argsort()]
    return (sorted_list)

In [ ]:
# load dataset from previous program 
TOF1_sample_dataset = np.load('TOF1_simulation_dataset.npy')

# helper function that will randomly select N patients from the dataset 
# returns a list of N show probabilities, from least to greatest 
def TOF1_patient_selection(N):
    indices = np.random.choice(len(TOF1_sample_dataset), N, replace = True)
    selected = TOF1_sample_dataset[indices]
    
    # sort everything by the show probabilities in increasing order
    sorted_list = selected[selected[:, 2].argsort()]
    return (sorted_list)

## Strategy Simulation Functions

### TOF

In [ ]:
def TOF0(schedule, N, F, iters):
    avg_show_rate = 1 - 0.2018964550044335
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(avg_show_rate)        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
    return it_list, ot_list, p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list

In [ ]:
def TOF1(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = TOF1_patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
           # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
         
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
    return it_list, ot_list, p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list

In [ ]:
def TOF2(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
    return it_list, ot_list, p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list

### UOF

In [ ]:
def UOF_S(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []

    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
    
    # additional metrics for UOF
    expected_shows_list = []
    expected_shows_m_list = []
    expected_shows_f_list = []
    expected_shows_s_list = []
    expected_shows_ns_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    #print("temp waiting", temp_waiting)
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        # Number of shows is equal to the number of patients whose wait time is not nan
        expected_shows_list.append(~np.sum(np.isnan(p_wait)))
        
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
        # flag for determining the group with max wait time (USING SCHOLARSHIP FOR NOW)
        scholarship_wait_isLonger = 0
        if (np.mean(scholarship_p_wait_list) > np.mean(noscholarship_p_wait_list)):
            scholarship_wait_isLonger == 1
        else:
            scholarship_wait_isLonger == 0
    return it_list, ot_list, p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list, expected_shows_list, scholarship_wait_isLonger 

In [ ]:
def UOF_MM(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    total_waiting_times = []

    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
    
    # additional metrics for UOF
    expected_shows_list = []
    expected_shows_m_list = []
    expected_shows_f_list = []
    expected_shows_s_list = []
    expected_shows_ns_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    #print("temp waiting", temp_waiting)
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        total_waiting_times.append(p_wait)
        
        # Number of shows is equal to the number of patients whose wait time is not nan
        expected_shows_list.append(~np.sum(np.isnan(p_wait)))

        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
        
    longestWait = np.max(np.nanmean(total_waiting_times, axis = 0))
    #print(longestWait)
    return it_list, ot_list, p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list, expected_shows_list, longestWait

### TOFF

In [ ]:
# TOFF_Range
def TOFF_R(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
                
    patient_range = np.nanmax(p_wait_list) - np.nanmin(p_wait_list)
    return it_list, ot_list, p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list, patient_range

In [ ]:
# TOFF_Variance
def TOFF_V(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        

        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
                
    patient_std = np.nanstd(p_wait_list)
    return it_list, ot_list, p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list, patient_std ** 2

### Find Optimal Schedule

In [ ]:
def find_optimal(strategy, N, F, rho, tao, omega, phi_r, phi_v, iters, print_bool):
    
    schedule_performance = []
    schedule_performance_std = []
    possible_schedules = all_schedules(N, F)
    shows = []
    
    # for UOF
    s_wait_isLonger = []
    ns_wait_isLonger = []
    longestWait = []

    # for TOFF
    patient_range = []
    patient_var = []
    
    for s in possible_schedules: 
        # randomly select 20% of schedules to examine
        if(np.random.rand() < 0.2):
            if(strategy == "TOF2"):
                it, ot, wt, scholarship_wt, noscholarship_wt = (TOF2(s, N, F, iters))
            if(strategy == "TOF1"):
                it, ot, wt, scholarship_wt, noscholarship_wt = (TOF1(s, N, F, iters))    
            if(strategy == "TOF0"):
                it, ot, wt, scholarship_wt, noscholarship_wt = (TOF0(s, N, F, iters))

            if(strategy == "UOF_S"):
                it, ot, wt, scholarship_wt, noscholarship_wt, E_shows, s_isLonger = (UOF_S(s, N, F, iters))
                shows.append(np.mean(E_shows))
                s_wait_isLonger.append(s_isLonger)
                ns_wait_isLonger.append(1 - s_isLonger)

            if(strategy == "UOF_MM"):
                it, ot, wt, scholarship_wt, noscholarship_wt, E_shows, lW = (UOF_MM(s, N, F, iters))
                shows.append(np.mean(E_shows))
                longestWait.append(lW)

            if(strategy == "TOFF_R"):
                it, ot, wt, scholarship_wt, noscholarship_wt, p_range = (TOFF_R(s, N, F, iters))
                patient_range.append(p_range)

            if(strategy == "TOFF_V"):
                it, ot, wt, scholarship_wt, noscholarship_wt, p_var = (TOFF_V(s, N, F, iters))
                patient_var.append(p_var)

            schedule_performance.append([np.mean(it), np.mean(ot), np.mean(wt), np.mean(scholarship_wt), np.mean(noscholarship_wt)])
            schedule_performance_std.append([np.std(it), np.std(ot), np.std(wt), np.std(scholarship_wt), np.std(noscholarship_wt)])

    # calcuate optimal based on objective function 
    if(strategy == "TOF2" or strategy == "TOF0" or strategy == "TOF1"):
        objective = []
        objective = [rho*schedule_performance[x][0] + tao*schedule_performance[x][1] + omega*schedule_performance[x][2] for x in range(len(schedule_performance))]

    elif(strategy == "UOF_S"):
        #print(len(schedule_performance))
        W_max = []
        for i in range(len(schedule_performance)):
            #print(i)
            if (s_wait_isLonger[i] == 1):
                W_max.append(shows[i] * schedule_performance[i][3])
            else:
                W_max.append(shows[i] * schedule_performance[i][4])
             
        objective = [rho*schedule_performance[x][0] + tao*schedule_performance[x][1] + omega*W_max[x] for x in range(len(schedule_performance))]
    
    elif(strategy == "UOF_MM"):
        W_max = []
        for i in range(len(schedule_performance)):
            W_max.append(shows[i] * longestWait[i])
        objective = [rho*schedule_performance[x][0] + tao*schedule_performance[x][1] + omega*W_max[x] for x in range(len(schedule_performance))]
    
    elif(strategy == "TOFF_R"):
        objective = []
        for i in range(len(schedule_performance[0])):
            o = rho * schedule_performance[i][0] + tao * schedule_performance[i][1] + omega * schedule_performance[i][2] + phi_r * patient_range[i]
            objective.append(o)
    
    elif(strategy == "TOFF_V"):
        objective = []
        for i in range(len(schedule_performance[0])):
            o = rho * schedule_performance[i][0] + tao * schedule_performance[i][1] + omega * schedule_performance[i][2] + phi_v * patient_var[i]
            objective.append(o)
    
    opt = np.argmin(objective)
    if(print_bool):
        print(possible_schedules[opt])
        print(transform_schedule(possible_schedules[opt]))
        print(schedule_performance[opt])

    return (possible_schedules[opt], schedule_performance[opt])

## Model Evaluation

In [ ]:
# Given a scheduling strategy, compute certain metrics (using actul information about whether they showed up) for model comparison
# Important metrics: 
# overtime 
# idle time
# average wait
# scholarship wait, no scholarship wait
# welfare disparity 
# cost 
def strat_eval(N, F, schedule, iters, rho, tao, omega, base, print_bool):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_s = []
        G_ns = []

        for i in range(N):
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)      
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        show = patient_sample[:, 3]
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(schedule))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
    
    
    schedule_performance = [np.mean(it_list), np.mean(ot_list), np.mean(p_wait_list), np.mean(scholarship_p_wait_list), np.mean(noscholarship_p_wait_list)]
    schedule_performance.append(rho*schedule_performance[0] + tao*schedule_performance[1] + omega*schedule_performance[2])

    objective = rho*np.array(it_list) + tao*np.array(ot_list) + omega*np.array(p_wait_list)
    
    # Fairness of scholarship 
    if(print_bool):
        print("Scholarship average wait time: ", np.mean(scholarship_p_wait_list))
        print("No scholarship average wait time: ", np.mean(noscholarship_p_wait_list))
        print("Statistical significance test: ")
    bias_ptest = stats.ttest_ind(scholarship_p_wait_list, noscholarship_p_wait_list)
    if(print_bool):
        print(bias_ptest)
        if(bias_ptest[1] < 0.05):
            print("STATISTICALLY SIGNIFICANT")
        print()
    
    # statistical significance with base model 

    performance_ptest = stats.ttest_ind(objective, base)
    if(print_bool):
        print("Performance Test")
        print(performance_ptest)
        if(performance_ptest[1] < 0.05):
            print("STATISTICALLY SIGNIFICANT")
    return schedule_performance


In [ ]:
# Given a scheduling strategy, compute certain metrics (using actul information about whether they showed up) for model comparison
# Important metrics: 
# overtime 
# idle time
# average wait
# male wait, female wait, scholarship wait, no scholarship wait
# welfare disparity 
# cost 
def TOF2_eval(N, F, schedule, iters, rho, tao, omega, print_bool):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_s = []
        G_ns = []

        for i in range(N):
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)      
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        show = patient_sample[:, 3]
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(schedule))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        

        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
    
    
    # it, ot, wt, scholarship_wt, noscholarship_wt
    schedule_performance = [np.mean(it_list), np.mean(ot_list), np.mean(p_wait_list), np.mean(scholarship_p_wait_list), np.mean(noscholarship_p_wait_list)]
    schedule_performance.append(rho*schedule_performance[0] + tao*schedule_performance[1] + omega*schedule_performance[2])

    objective = rho*np.array(it_list) + tao*np.array(ot_list) + omega*np.array(p_wait_list)

    # Fairness of scholarship 
    if(print_bool):
        print("Scholarship average wait time: ", np.mean(scholarship_p_wait_list))
        print("No scholarship average wait time: ", np.mean(noscholarship_p_wait_list))
        print("Statistical significance test: ")
    ptest = stats.ttest_ind(scholarship_p_wait_list, noscholarship_p_wait_list)
    if(print_bool):
        print(ptest)
        if(ptest[1] < 0.05):
            print("STATISTICALLY SIGNIFICANT")
        

    
    return schedule_performance, objective


# Running Simulations

In [ ]:
def sensitivity_analysis(N, F, rho, tao, omega, phi_r, phi_v):
    # save results in df
    column_names = ["model_name", "idle_time", "overtime", "scholarship_wait", "no_scholarship_wait", "disparity", "TOF_obj", "cost"]
    results = pd.DataFrame(columns = column_names)
    
    iters = 500
    
    # find optimal schedules
    print("TOF2")
    TOF2_sched, TOF2_performance = find_optimal("TOF2", N, F, rho, tao, omega, phi_r, phi_v, iters, True)
    print()

    print("TOF0")
    TOF0_sched, TOF0_performance = find_optimal("TOF0", N, F, rho, tao, omega, phi_r, phi_v, iters, True)
    print()
    print("TOF1")
    TOF1_sched, TOF1_performance = find_optimal("TOF1", N, F, rho, tao, omega, phi_r, phi_v, iters, True)
    print()

    print("UOF_S")
    UOF_S_sched, UOF_S_performance = find_optimal("UOF_S", N, F, rho, tao, omega, phi_r, phi_v, iters, True)
    print()
    print("UOF_MM")
    UOF_MM_sched, UOF_MM_performance = find_optimal("UOF_MM", N, F, rho, tao, omega, phi_r, phi_v, iters, True)
    print()

    print("TOFF_R")
    TOFF_R_sched, TOFF_R_performance = find_optimal("TOFF_R", N, F, rho, tao, omega, phi_r, phi_v, iters, True)
    print()
    print("TOFF_V")
    TOFF_V_sched, TOFF_V_performance = find_optimal("TOFF_V", N, F, rho, tao, omega, phi_r, phi_v, iters, True)
    print()
    
    # evaluate model performance 
    print("TOF2")
    TOF2_base_comparison = TOF2_eval(N, F, TOF2_sched, iters, rho, tao, omega, True)
    print(TOF2_base_comparison[0])
    d = (TOF2_base_comparison[0][3] - TOF2_base_comparison[0][4])/TOF2_base_comparison[0][4]
    df_row = {'model_name': "TOF2", 'idle_time': TOF2_base_comparison[0][0], 'overtime': TOF2_base_comparison[0][1], 'scholarship_wait': TOF2_base_comparison[0][3], 'no_scholarship_wait': TOF2_base_comparison[0][4], 'disparity': d, 'TOF_obj': TOF2_base_comparison[0][5], 'cost': 0}
    results = results.append(df_row, ignore_index = True)
    
    opt_cost = TOF2_base_comparison[0][5]
    base = TOF2_base_comparison[1]
    print()
    
    names = ["TOF1", "TOF0", "UOF-NS", "UOF-MM", "TOFF_R", "TOFF_V"]
    opt_scheds = [TOF1_sched, TOF0_sched, UOF_S_sched, UOF_MM_sched, TOFF_R_sched, TOFF_V_sched]
    
    for i in range(len(opt_scheds)): 
        #base = TOF2_base_comparison[1]
        print(names[i])
        print()
        output = strat_eval(N, F, opt_scheds[i], iters, rho, tao, omega, base, True)
        d = (output[3] - output[4])/output[4]
        c = (output[5] - opt_cost)/opt_cost
        df_row = {'model_name': names[i], 'idle_time': output[0], 'overtime': output[1], 'scholarship_wait': output[3], 'no_scholarship_wait': output[4], 'disparity': d, 'TOF_obj': output[5], 'cost': c}
        results = results.append(df_row, ignore_index = True)
    
    return results
    


In [ ]:
np.random.seed(0)

# idle time cost 
rho = 1
# overtime cost 
tao = 1.5
# waiting time cost
omega = 0.5

# fairness cost 
phi_r = 1.50
phi_v = 1.50

In [ ]:
n6f4_results = sensitivity_analysis(6, 4, rho, tao, omega, phi_r, phi_v)
print(n6f4_results)

# Tuning Fairness Coefficient

In [ ]:
np.random.seed(100)

iters = 500
# number of appointment requests
N = 6
# number of appointment slots 
F = 4

# idle time cost 
rho = 1
# overtime cost 
tao = 1.5
# waiting time cost
omega = 0.5

# NEED TO TUNE THIS PARAMETER 
# fairness cost 
phi_r = 0.75
phi_v = 1.25
phi = np.arange(0, 2.1, 0.10)

column_names = ["phi", "model_name", "scholarship_wait", "no_scholarship_wait", "TOF_obj"]
phi_df = pd.DataFrame(columns = column_names)

In [ ]:
for p in phi:
    #print(p)
    # find optimal given the weights 
    TOF2_sched, TOF2_performance = find_optimal("TOF2", N, F, rho, tao, omega, p, p, iters, False)
    TOFF_R_sched, TOFF_R_performance = find_optimal("TOFF_R", N, F, rho, tao, omega, p, p, iters, False)
    TOFF_V_sched, TOFF_V_performance = find_optimal("TOFF_V", N, F, rho, tao, omega, p, p, iters, False)
    
    # add evaluation metrics to a dataframe 
    TOF2_base_comparison = TOF2_eval(N, F, TOF2_sched, iters, rho, tao, omega, False)
    TOF2_row = {'phi': p, 'model_name': 'TOF2', 'scholarship_wait': TOF2_base_comparison[0][3], 'no_scholarship_wait': TOF2_base_comparison[0][4], 'TOF_obj': TOF2_base_comparison[0][5]}
    phi_df = phi_df.append(TOF2_row, ignore_index = True)
    base = TOF2_base_comparison[1]
    
    TOFF_R_eval = strat_eval(N, F, TOFF_R_sched, iters, rho, tao, omega, False)
    TOFF_R_row = {'phi': p, 'model_name': 'TOFF_R', 'scholarship_wait': TOFF_R_eval[3], 'no_scholarship_wait': TOFF_R_eval[4], 'TOF_obj': TOFF_R_eval[5]}
    phi_df = phi_df.append(TOFF_R_row, ignore_index = True)
    
    TOFF_V_eval = strat_eval(N, F, TOFF_V_sched, iters, rho, tao, omega, False)
    TOFF_V_row = {'phi': p, 'model_name': 'TOFF_V', 'scholarship_wait': TOFF_V_eval[3], 'no_scholarship_wait': TOFF_V_eval[4], 'TOF_obj': TOFF_V_eval[5]}
    phi_df = phi_df.append(TOFF_V_row, ignore_index = True)
    

In [ ]:
#print(phi_df)
TOF2_results = phi_df.loc[phi_df['model_name'] == 'TOF2']
TOFF_R_results = phi_df.loc[phi_df['model_name'] == 'TOFF_R']
TOFF_V_results = phi_df.loc[phi_df['model_name'] == 'TOFF_V']

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8))
plt.plot(phi, TOF2_results.scholarship_wait, 'g')
plt.plot(phi, TOF2_results.no_scholarship_wait, 'g--')
plt.plot(phi, TOFF_R_results.scholarship_wait, 'b')
plt.plot(phi, TOFF_R_results.no_scholarship_wait, 'b--')
plt.plot(phi, TOFF_V_results.scholarship_wait, 'r')
plt.plot(phi, TOFF_V_results.no_scholarship_wait, 'r--')
plt.title("Scholarship and No-Scholarship Expected Wait Times")
plt.xlabel("phi")
plt.ylabel("Expected Wait Time")
plt.legend(["TOF2 Scholarship", "TOF2 No Scholarship", "TOFF_R Scholarship", "TOFF_R No Scholarship", "TOFF_V Scholarship", "TOFF_V No Scholarship"])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(phi, TOF2_results.scholarship_wait - TOF2_results.no_scholarship_wait, 'g')
m_1, b_1 = np.polyfit(phi, TOF2_results.scholarship_wait - TOF2_results.no_scholarship_wait, 1)
plt.plot(phi, m_1 * phi + b_1, 'g--')

plt.plot(phi, TOFF_R_results.scholarship_wait - TOFF_R_results.no_scholarship_wait, 'b')
m_2, b_2 = np.polyfit(phi, TOFF_R_results.scholarship_wait - TOFF_R_results.no_scholarship_wait, 1)
plt.plot(phi, m_2 * phi + b_2, 'b--')

plt.plot(phi, TOFF_V_results.scholarship_wait - TOFF_V_results.no_scholarship_wait, 'r')
m_3, b_3 = np.polyfit(phi, TOFF_V_results.scholarship_wait - TOFF_V_results.no_scholarship_wait, 1)
plt.plot(phi, m_3 * phi + b_3, 'r--')

plt.title("Difference Between Scholarship Wait and No-Scholarship Wait Times")
plt.xlabel("phi")
plt.ylabel("Scholarship Wait - No Scholarship Wait")
plt.legend(["TOF2", "TOF2 LOBF", "TOFF_R", "TOFF_R LOBF", "TOFF_V", "TOFF_V LOBF"], loc = "upper right")

In [ ]:
plt.figure(figsize=(12,8))

plt.plot(phi, TOF2_results.TOF_obj, 'g')
m_1, b_1 = np.polyfit(phi, TOF2_results.TOF_obj, 1)
plt.plot(phi, m_1 * phi + b_1, 'g--')

plt.plot(phi, TOFF_R_results.TOF_obj, 'b')
m_2, b_2 = np.polyfit(phi, TOFF_R_results.TOF_obj, 1)
plt.plot(phi, m_2 * phi + b_2, 'b--')

plt.plot(phi, TOFF_V_results.TOF_obj, 'r')
m_3, b_3 = np.polyfit(phi, TOFF_V_results.TOF_obj, 1)
plt.plot(phi, m_3 * phi + b_3, 'r--')

plt.title("TOF Objective Function Results")
plt.xlabel("phi")
plt.legend(["TOF2", "TOF2 LOBF", "TOFF_R", "TOFF_R LOBF", "TOFF_V", "TOFF_V LOBF"])